# Exercise

In [3]:
#1 
#Load boston setting X as boston.data and y as boston.target
#Attempt the grid search using polyregression + (linear, ridge, lasso, elastic net), and 
#Does feature mechanisms on ridge/lasso/elastic helps here?
#why do you think the result is like this?
#what is the value of alpha, and what does it means?
#YOURE CODE HERE

# Solution

In [ ]:
#1
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error, r2_score
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

import pandas as pd
import numpy as np

#we know in advance that ElasticNet gonna complain about
#needing more iterations.  Unfortunately, to prevent my pc 
#from crashing, I will simply ignore this warning, and
#likely set the tol to a bit high
warnings.filterwarnings("ignore", category=ConvergenceWarning,
                        module="sklearn")

boston = load_boston()
X = boston.data
y = boston.target
X = scaler.fit_transform(X)  #by scaling it helps reach convergence faster

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size = 0.3, random_state=5)

params_linear = {'polynomialfeatures__degree': np.arange(1, 10)}
linear = make_pipeline(PolynomialFeatures(), LinearRegression(normalize=True))
params_Ridge = {'polynomialfeatures__degree': np.arange(1, 10),
                'ridge__alpha': [0.1,0.01,0.001,0.0001]}
ridge = make_pipeline(PolynomialFeatures(), Ridge(normalize=True))  
params_Lasso = {'polynomialfeatures__degree': np.arange(1, 10),
                'lasso__alpha': [1,0.1,0.01,0.001,0.0001]}
lasso = make_pipeline(PolynomialFeatures(), 
                      Lasso(normalize=True, tol=0.01, max_iter=100000))
params_Elasticnet = {'polynomialfeatures__degree': np.arange(1, 10),
                'elasticnet__alpha': [1,0.1,0.01,0.001,0.0001],
                "elasticnet__l1_ratio": np.linspace(0, 1, 5)}
elastic = make_pipeline(PolynomialFeatures(), 
                      ElasticNet(normalize=True, tol=0.1, max_iter=100000))

params = [params_linear, params_Ridge, params_Lasso, params_Elasticnet]
models = [linear, ridge, lasso, elastic]
features_name = ['linearregression', 'ridge', 'lasso', 'elasticnet']

for ix, model in enumerate(models):
    cv = ShuffleSplit(n_splits = 10, test_size = 0.2, random_state=42)
    print(model)
    grid = GridSearchCV(model, params[ix], cv=cv)
    
    #grid.fit will fit the model at each grid point
    grid.fit(X_train, y_train)

    #print the best parameters
    print("Best params: ", grid.best_params_)

    #make prediction
    model = grid.best_estimator_
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    #print the stats
    print("Coefficients: ", model.named_steps[features_name[ix]].coef_)
    print(f"r^2 = {r2_score(y_test, y_pred):.3f}")
    print(f"MSE = {mean_squared_error(y_test, y_pred):.2f}")
    n, p = X.shape[0], X.shape[1]
    adjusted_rsqrt = 1-(1-r2_score(y_test, y_pred))*(n-1)/(n-p-1)
    print(f"adjusted $r^2$ = {adjusted_rsqrt:.3f}")


Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('linearregression', LinearRegression(normalize=True))])
Best params:  {'polynomialfeatures__degree': 1}
Coefficients:  [ 0.         -1.32750493  0.96447433 -0.17391979  0.19945597 -1.49757922
  2.83543215 -0.2962689  -2.80831532  2.76854145 -2.12866545 -2.11368262
  1.15569647 -3.29628098]
r^2 = 0.677
MSE = 30.70
adjusted $r^2$ = 0.669
Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('ridge', Ridge(normalize=True))])
Best params:  {'polynomialfeatures__degree': 2, 'ridge__alpha': 0.01}
Coefficients:  [ 0.00000000e+00 -1.44816043e-01 -5.64496705e-02  3.98000454e-01
  4.38845564e-01 -8.51675780e-01  3.55010742e+00 -1.37778256e+00
 -1.86222774e+00  1.10837142e+00 -1.06506148e+00 -8.12821444e-01
  1.25327660e+00 -3.26288191e+00  1.63850840e-01  4.71665252e-01
 -1.16864891e-01  2.94839213e+00 -6.22042398e-01  3.06562044e-01
 -4.06958944e-01  3.23915966e-01 -5.05610073e-01  7.88507767